In [27]:
import numpy as np 
import matplotlib as plt
import pandas as pd
import seaborn as sns
import scipy

In [49]:
penguins_df = pd.read_csv("noisy_datasets/penguins.csv")
diamonds_df = pd.read_csv("noisy_datasets/diamond.csv")
epicurious_df = pd.read_csv("noisy_datasets/epicurious.csv")

print("Penguin Data \n", penguins_df.describe())
print("\nDiamond Data \n", diamonds_df.describe())
print("\nEpicurious Dataset\n", epicurious_df.describe())

Penguin Data 
        calorie requirement  average sleep duration  bill_length_mm  \
count           344.000000              344.000000      337.000000   
mean           5270.002907               10.447674       45.494214   
std            1067.959116                2.265895       10.815787   
min            3504.000000                7.000000       32.100000   
25%            4403.000000                9.000000       39.500000   
50%            5106.500000               10.000000       45.100000   
75%            6212.750000               12.000000       49.000000   
max            7197.000000               14.000000      124.300000   

       bill_depth_mm  flipper_length_mm  body_mass_g         year  
count     333.000000         336.000000   339.000000   342.000000  
mean       18.018318         197.764881  4175.463127  2008.035088  
std         9.241384          27.764491   858.713267     0.816938  
min        13.100000          10.000000   882.000000  2007.000000  
25%        15.

In [51]:
penguins_df = penguins_df.dropna(subset=['species', 'island', 'gender'])
penguins_df = penguins_df.fillna(penguins_df.select_dtypes(include='number').mean())
penguins_df[['species', 'island', 'gender']] = penguins_df[['species', 'island', 'gender']].applymap(str.lower)
z = np.abs(scipy.stats.zscore(penguins_df['calorie requirement']))
z = np.abs(scipy.stats.zscore(penguins_df['average sleep duration']))
threshold = 1.6
outliers_cr = penguins_df['calorie requirement'][z > threshold]
outliers_asd = penguins_df['average sleep duration'][z > threshold]
outliers_bl = penguins_df['bill_length_mm'][z > threshold]
outliers_bd = penguins_df['bill_depth_mm'][z > threshold]
outliers_fl = penguins_df['flipper_length_mm'][z > threshold]
outliers_bm = penguins_df['body_mass_g'][z > threshold]
print(outliers_cr.count())
print(outliers_asd.count())


0
0
